In [4]:
%cd ..

/home/sguo/code/BDC_Chatbot


/home/sguo/code/BDC_Chatbot/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:

from pathlib import Path
from tqdm import tqdm
import pickle

from datetime import datetime

from preproc.proc_BDC_repo import get_fellow_files, get_data_mdx_files, clean_mdx, get_all_mdx_paths, clean_path


# process data dir

data_dir = "../../interim-bdc-website/src/data/"



# region: process fellows
print("Processing fellows...")
fellow_dir = data_dir + "fellows/"

fellows = get_fellow_files(Path(fellow_dir).resolve(), data_dir)

for fellow in fellows:
    fellow['metadata']['project_title'] = fellow['metadata']['project']['title']
    fellow['metadata']['project_abstract'] = fellow['metadata']['project']['abstract']
    del fellow['metadata']['project']





print(f"Fellows: {len(fellows)}")

# process latest-updates
print("Processing latest-updates...")
updates_dir = data_dir + "latest-updates/"

updates = get_data_mdx_files(Path(updates_dir).resolve(), data_dir)

for update in updates:
    

    update['metadata']['timestamp'] = datetime.combine(update['metadata']['date'], datetime.min.time().replace(hour=12)).timestamp()
    update['metadata']['date'] = update['metadata']['date'].strftime("%Y-%m-%d")
    
    update['metadata']['tags'] = ", ".join(update['metadata']['tags'])
    # del value if it is a dictionary or list of dictionaries
    del_keys = []
    for key, value in update['metadata'].items():
        if isinstance(value, dict):
            del_keys.append(key)
        elif isinstance(value, list):
            if isinstance(value[0], dict):
                del_keys.append(key)

    for key in del_keys:
        del update['metadata'][key]
    
    
    
# process events
print("Processing events...")
events_dir = data_dir + "events/"

events = get_data_mdx_files(Path(events_dir).resolve(), data_dir)


# convert event date to string
for event in events:
    event['metadata']['timestamp'] = datetime.combine(event['metadata']['date'], datetime.min.time().replace(hour=12)).timestamp()
    event['metadata']['date'] = event['metadata']['date'].strftime("%Y-%m-%d")
    
    
    event['metadata']['tags'] = ", ".join(event['metadata']['tags'])
    
    del_keys = []
    for key, value in event['metadata'].items():
        if isinstance(value, dict):
            del_keys.append(key)
        elif isinstance(value, list):
            if isinstance(value[0], dict):
                del_keys.append(key)
    for key in del_keys:
        del event['metadata'][key]
    
    
    
# endregion


# region: process pages dir


pages_dir = '../../interim-bdc-website/src/pages/'
page_dir_paths = ["use-bdc/analyze-data/", ]
page_file_paths = ["join-bdc/index.mdx", "use-bdc/share-data.mdx", 
                   "user-resources/terms-of-use.mdx", "user-resources/usage-costs.mdx", "user-resources/usage-terms.mdx",
                   "about/key-collaborations.mdx", "about/overview.mdx", "about/research-communities.mdx",
                   "use-bdc/explore-data/index.mdx"]


mdx_paths = get_all_mdx_paths(pages_dir, page_dir_paths, page_file_paths)


metadata_list = []
page_content_list = []

for path in tqdm(mdx_paths):
    header, page_content = clean_mdx(path)
    header['file_path'] = clean_path(path, pages_dir)
    metadata_list.append(header)
    page_content_list.append(page_content)

# Save pages data
pages_data = []
for metadata, content in tqdm(zip(metadata_list, page_content_list)):
    if 'menu' in metadata.keys():
        metadata['headings'] = ', '.join([pair['heading'] for pair in metadata['menu']])
        metadata['hrefs'] = ', '.join([pair['href'] for pair in metadata['menu']])
        del metadata['menu']
    
    
    pages_data.append({'metadata': metadata, 'content': content})
    
    
    
# endregion








Processing fellows...


Reading fellow files: 100%|██████████| 47/47 [00:00<00:00, 772.41it/s]


Fellows: 47
Processing latest-updates...


Parsing MDX files: 100%|██████████| 8/8 [00:00<00:00, 735.78it/s]


Processing events...


100%|██████████| 11/11 [00:00<00:00, 835.38it/s]
11it [00:00, 37571.13it/s]


In [7]:
print(fellows[0]['metadata']['file_path'])
print(updates[0]['metadata']['file_path'])
print(events[0]['metadata']['file_path'])
print(pages_data[0]['metadata']['file_path'])


interim-bdc-website/src/data/fellows/gaynor-sheila.md
interim-bdc-website/src/data/latest-updates/2023-09-11_SCDdatasets/index.mdx
interim-bdc-website/src/data/events/2023-02-22_Community-Hours.mdx
interim-bdc-website/src/pages/use-bdc/analyze-data/index.mdx


In [8]:
# save all content and metadata in pkl
# Save fellows data
with open('../data/fellows.pkl', 'wb') as f:
    pickle.dump(fellows, f)

# Save latest updates data
with open('../data/latest_updates.pkl', 'wb') as f:
    pickle.dump(updates, f)

# Save events data  
with open('../data/events.pkl', 'wb') as f:
    pickle.dump(events, f)


with open('../data/pages.pkl', 'wb') as f:
    pickle.dump(pages_data, f)